# Investigate normal expression of cis proteins that show equivalence between samples with and without the event

We want to know why these proteins don't show a significant change even when the arm is deleted. It's possible that they have low expression to begin with, so the change from the deletion isn't noticeable.

Unfortunately, we can only get relative expression levels from the CPTAC data, not absolute expression levels. So instead of using CPTAC dat, we're going to be working with tissue-specific absolute expression data from this paper: Wang D, Eraslan B, Wieland T, et al. A deep proteome and transcriptome abundance atlas of 29 healthy human 
tissues. Mol Syst Biol. 2019;15(2):e8503. Published 2019 Feb 18. doi:10.15252/msb.20188503

We downloaded the specific table, Table 5, from https://www.embopress.org/action/downloadSupplement?doi=10.15252%2Fmsb.20188503&file=msb188503-sup-0007-TableEV5.zip

In [1]:
import pandas as pd
import numpy as np
import os
import cnvutils
import scipy.stats
import altair as alt
from toolz import pipe

In [2]:
alt.data_transformers.disable_max_rows()

def json_dir(data, data_dir):
    os.makedirs(data_dir, exist_ok=True)
    return pipe(data, alt.to_json(filename=os.path.join(data_dir, "{prefix}-{hash}.{extension}")) )

alt.data_transformers.register("json_dir", json_dir)
alt.data_transformers.enable("json_dir", data_dir="plot_data")

DataTransformerRegistry.enable('json_dir')

In [3]:
CHROMOSOME = '8'
ARM = 'q'
CIS_OR_TRANS = "cis"

## Transform to log(x + 1), and exclude zeros

Based on the plots in the 10_normal_dist_exploration notebook, it looks like our best option is to use a log(x + 1) scale, and exclude zeros. From a biological perspective, I'm fine with excluding the zeros, because proteins that aren't expressed seem a different class from proteins that just have low expression. Although it is important to remember that it's also possible that proteins with zero copies may have just been too low in expression to be detected. Nevertheless, the fact that there's such a huge number of proteins with zero copies suggests that they aren't just the few escaping detection.

In [4]:
expr = cnvutils.get_normal_expr_table()

expr_long = expr.\
drop(
    columns=["Gene_ID", "Protein_ID"]
).\
melt(
    id_vars="Gene_name",
    var_name="tissue_type",
    value_name="prot_copy_count"
)

expr_plus1_log10 = expr_long.assign(prot_copy_count=np.log10(expr_long["prot_copy_count"] + 1))
expr_transf = expr_plus1_log10[expr_plus1_log10["prot_copy_count"] > 0]

## Calculate "low" cutoff for each tissue type

In [5]:
low_cutoffs = expr_transf.groupby("tissue_type").quantile(0.25)
low_cutoffs

,prot_copy_count
tissue_type,
Adrenal gland,3.639864
Appendix,3.580872
Brain,3.593914
Colon,3.513825
Duodenum,3.851583
Endometrium,3.617557
Esophagus,3.797054
Fallopian tube,3.501528
Fat,3.843306


## Compare expression of equivalent genes

In [6]:
equiv_prots = pd.read_csv(f"pancancer_summary_equiv_{CHROMOSOME}{ARM}_{CIS_OR_TRANS}.tsv", sep="\t")
equiv_prots

,protein,cancers,num_cancers
0,INTS8,brca_colon_hnscc_lscc_ovarian,5
1,NCALD,brca_colon_hnscc_lscc_ovarian,5
2,EMC2,brca_colon_lscc_luad_ovarian,5
3,RPL30,colon_hnscc_lscc_luad_ovarian,5
4,DEPTOR,brca_colon_hnscc_lscc,4
5,FABP5,brca_colon_luad_ovarian,4
6,TBC1D31,brca_hnscc_lscc_luad,4
7,ASAP1,colon_hnscc_lscc_luad,4
8,PAG1,colon_hnscc_lscc_ovarian,4
9,CA3,colon_hnscc_luad_ovarian,4


In [7]:
commonly_equiv = equiv_prots[equiv_prots["num_cancers"] >= 3]
sel_expr = expr.loc[
    expr["Gene_name"].isin(commonly_equiv["protein"]),
    ["Gene_name", "Protein_ID", "Colon", "Esophagus", "Lung", "Ovary"]
].\
set_index(["Gene_name", "Protein_ID"]).\
transpose()

sel_expr.columns = sel_expr.columns.to_flat_index()

# Transform the data to log10(x + 1), join in cutoffs column
sel_expr = sel_expr.\
apply(lambda x: np.log10(x + 1), axis="columns").\
join(low_cutoffs, how="left")

sel_expr = sel_expr.apply(lambda x: x <= sel_expr["prot_copy_count"]).\
drop(columns="prot_copy_count").\
transpose()

sel_expr.index = pd.MultiIndex.from_tuples(sel_expr.index).droplevel(1)

sel_expr

,Colon,Esophagus,Lung,Ovary
ASAP1,False,False,False,False
CA3,False,False,False,False
COL14A1,False,False,False,False
COX6C,False,False,False,False
CPQ,False,False,False,False
DEPTOR,False,False,False,False
DSCC1,False,True,False,False
EMC2,False,False,False,False
FABP4,False,False,False,False
FABP5,False,False,False,False


In [8]:
sel_expr.shape

(26, 4)

In [9]:
sel_expr.sum(axis="index")

Colon        7
Esophagus    7
Lung         5
Ovary        5
dtype: int64

### Plot on the distribution

In [10]:
def plot_cancer_type_normal_dist(cancer_type, tissue_type, expr_data, sel_genes):
    
    # Select the expression data for our chosen tissue type
    expr_data = expr_data[expr_data["tissue_type"] == tissue_type]
    
    dist = alt.Chart().mark_bar().encode(
        x=alt.X(
            "prot_copy_count",
            bin=alt.Bin(step=0.25),
            title="Protein copy counts (log transformed)"
        ),
        y=alt.Y(
            "count()",
            title="Number of proteins"
        )
    ).properties(
        width=800
    )

    # Add vertical lines at the expression levels of the proteins that didn't show a cis effect
    lines_no_cis = [
        alt.Chart().mark_rule(color=alt.Value("#d10000")).encode(
            x=f"{gene}:Q"
        )

        for gene in sel_genes
    ]

    # Get the expression levels for those proteins
    expr_no_cis = expr_data.\
    loc[expr_data["Gene_name"].isin(sel_genes)].\
    set_index("Gene_name")["prot_copy_count"].\
    astype(str).\
    to_dict()

    # Layer all the charts together
    chart = alt.layer(
        dist,
        *lines_no_cis,
        data=expr_data
    ).transform_calculate(
        **expr_no_cis
    ).properties(
        title=cancer_type
    )
    
    return chart

In [11]:
cancer_tissue = {
    "colon": "Colon",
    "hnscc": "Esophagus",
    "luad": "Lung",
    "lscc": "Lung",
    "ovarian": "Ovary"
}

alt.vconcat(*[
    plot_cancer_type_normal_dist(
        cancer_type=cancer, 
        tissue_type=tissue,
        expr_data=expr_transf,
        sel_genes=sel_expr.index.tolist()
    )
    
    for cancer, tissue in cancer_tissue.items()
]).properties(
    title=f"Normal expression of proteins that showed no cis effect for {CHROMOSOME}{ARM}"
).configure_title(
    anchor="middle",
    fontSize=14
)

alt.VConcatChart(...)

## Look at cancers individually

In [12]:
split_equiv = pd.\
read_csv(f"{CHROMOSOME}{ARM}_{CIS_OR_TRANS}_equiv.tsv", sep="\t").\
rename(columns={"Name": "protein"}).\
set_index("protein")

cancer_types = sorted(split_equiv.columns.to_series().str.split("_", n=1, expand=True)[0].unique())

long_split_equiv = pd.DataFrame()

for cancer_type in cancer_types:
    cancer_df = split_equiv.\
    loc[:, split_equiv.columns.str.startswith(cancer_type)].\
    dropna(axis="index", how="all").\
    reset_index(drop=False)
    
    # If the cancer type has database IDs, make a separate column that has them.
    # If not, create a column of NaNs (so that the tables all match)
    if f"{cancer_type}_Database_ID" in cancer_df.columns:
        cancer_df = cancer_df.rename(columns={f"{cancer_type}_Database_ID": "Database_ID"})
    else:
        cancer_df = cancer_df.assign(Database_ID=np.nan)
        
    # Rename the pvalue and diff columns to not have the cancer type
    cancer_df = cancer_df.rename(columns={
        f"{cancer_type}_pvalue": "adj_p",
        f"{cancer_type}_diff": "change"
    }).\
    assign(cancer_type=cancer_type)
    
    # Reorder the columns
    cancer_df = cancer_df[["cancer_type", "protein", "Database_ID", "adj_p"]]
    
    # Append to the overall dataframe
    long_split_equiv = long_split_equiv.append(cancer_df)

# Drop duplicate rows and reset the index
long_split_equiv = long_split_equiv[~long_split_equiv.duplicated(keep=False)].\
reset_index(drop=True)

long_split_equiv

,cancer_type,protein,Database_ID,adj_p
0,brca,ANKRD46,NP_001257308.1|NP_001257307.1,0.195904
1,brca,ASAP1,NP_060952.2|NP_001234925.1,0.136409
2,brca,ATAD2,NP_054828.2,0.999982
3,brca,ATP6V1C1,NP_001686.1,0.974040
4,brca,C8orf37,NP_808880.1,0.103086
5,brca,CA13,NP_940986.1,0.103086
6,brca,CA3,NP_005172.1,0.089796
7,brca,CHMP4C,NP_689497.1,0.103086
8,brca,CNGB3,NP_061971.3,0.399696
9,brca,COL14A1,NP_066933.1,0.999982


In [13]:
cancer_tissue = {
    "colon": "Colon",
    "hnscc": "Esophagus",
    "luad": "Lung",
    "lscc": "Lung",
    "ovarian": "Ovary"
}

cancer_expr = {}

for cancer_type, tissue_type in cancer_tissue.items():
    
    cancer_equiv = long_split_equiv[long_split_equiv["cancer_type"] == cancer_type]
    sel_expr = expr.loc[
        expr["Gene_name"].isin(cancer_equiv["protein"]),
        ["Gene_name", "Protein_ID", tissue_type]
    ].\
    set_index(["Gene_name", "Protein_ID"]).\
    transpose()

    sel_expr.columns = sel_expr.columns.to_flat_index()

    # Transform the data to log10(x + 1), join in cutoffs column
    sel_expr = sel_expr.\
    apply(lambda x: np.log10(x + 1), axis="columns").\
    join(low_cutoffs, how="left")

    sel_expr = sel_expr.apply(lambda x: x <= sel_expr["prot_copy_count"]).\
    drop(columns="prot_copy_count").\
    transpose()

    sel_expr.index = pd.MultiIndex.from_tuples(sel_expr.index).droplevel(1)
    
    print(f"{cancer_type}: {sel_expr.sum()[0]}/{sel_expr[tissue_type].size} ({sel_expr.sum()[0] / sel_expr[tissue_type].size:.2f}) below cutoff")

    cancer_expr[cancer_type] = sel_expr

colon: 13/72 (0.18) below cutoff
hnscc: 26/87 (0.30) below cutoff
luad: 18/91 (0.20) below cutoff
lscc: 18/91 (0.20) below cutoff
ovarian: 21/88 (0.24) below cutoff


In [14]:
cancer_expr["colon"]

,Colon
ASAP1,False
ATAD2,False
ATP6V1C1,False
ATP6V1C1,True
CA13,False
CA3,False
CHMP4C,False
COL14A1,False
COLEC10,True
COX6C,False


In [15]:
cancer_expr["hnscc"]

,Esophagus
ANKRD46,True
ASAP1,False
ATAD2,False
ATP6V1C1,False
ATP6V1C1,True
C8orf37,False
CA13,False
CA3,False
CHMP4C,True
COL14A1,False


In [16]:
cancer_expr["lscc"]

,Lung
ANKRD46,True
ASAP1,False
ATAD2,False
ATP6V1C1,False
ATP6V1C1,True
C8orf37,True
CA13,False
CA3,False
CHMP4C,False
COL14A1,False


In [17]:
cancer_expr["luad"]

,Lung
ANKRD46,True
ASAP1,False
ATAD2,False
ATP6V1C1,False
ATP6V1C1,True
C8orf37,True
CA13,False
CA2,False
CA3,False
CHMP4C,False


In [18]:
cancer_expr["ovarian"]

,Ovary
ANKRD46,True
ASAP1,False
ATAD2,False
ATP6V1C1,False
ATP6V1C1,True
C8orf37,False
CA13,False
CA3,False
CHMP4C,False
COL14A1,False
